# A. Imports and initial setup

## A.1 import data

In [1]:
import numpy as np 
import pandas as pd 

from utils import get_stopwords, get_stemmer, tokenizer, data2idx
dataset_location='../development.json'
# dataset_location='../CorpusTrabalho4.zip'
df = pd.read_json(dataset_location, orient='records')
# print(df.count()) 9998
df.sample(10).head(10)

,description,id,title
7418,Operador de estacionamento ( manobrista/ caixa...,7418,Vaga para Operador de Bate-Estacas
7814,WEB DESIGNER / CAMPINAS / SP / 2 VAGA(S) ...,7814,WEB DESIGNER / CAMPINAS / SP / 2 VAGA -S- - 13...
9478,Atividades Profissionais: Atuar com vendas ext...,9478,Vendedor Externo
4880,Conceituada empresa de calcados carioca admite...,4880,Vendedor
3853,Realizar manutencao preventiva e corretiva de ...,3853,Tecnico De Suporte
1504,Vendedor internoRequisitos:Ensino medio comple...,1504,Vaga para Vendedor Interno
805,\r\n Ira registrar os produtos ...,805,Operador(a) de Caixa
6092,Wea lab esta com 1 vaga(s) em aberto para hort...,6092,Vaga para Ajudante de Soldador
7125,Administracao e monitoramento de banco de dado...,7125,Analista de Tecnologia da Informacao Senior
7908,Empresa contrata urgente\nVaga :auxiliar de al...,7908,Vaga para Auxiliar de Almoxarifado


## A.2 Stopwords

In [2]:
stopwords= list(get_stopwords())
ind = np.random.randint(0,len(stopwords), size=5)
print('total stop words are %d' % (len(stopwords)))
print([stopwords[i] for i in ind])      

total stop words are 203
['for', 'do', 'estejamos', 'isto', 'do']


## A.3 Stemmer

In [3]:
stemmer= get_stemmer() 
words_test= ['Programador', 'Programa', 'pegaram', 'correram']
print('Before stemming:')
print(words_test)
print('After stemming:')
print([stemmer.stem(w) for w in words_test])

Before stemming:
['Programador', 'Programa', 'pegaram', 'correram']
After stemming:
['program', 'program', 'peg', 'corr']


## A.4 Tokenize the title

In [4]:
tokenfy = lambda x : tokenizer(x, stopwords= stopwords, stemmer=stemmer)
df['token_title'] = df['title'].transform(tokenfy)
df.sample(10).head(10)

,description,id,title,token_title
9787,AUXILIAR DE PRODUcaO / VALINHOS / SP / 1 VAGA(...,9787,assistente de PRODUcaO / VALINHOS / SP / 1 VAG...,assistent produca valinh sp 1 vag s 1392356
3847,Supervisor de armazem Ira supervisionar as ati...,3847,Supervisor de armazem,supervisor armaz
1840,enfermeiro edp zona sul / rj hospital na zona ...,1840,Vaga para Enfermeiro,vag enfermeir
6048,Empresa Contratante\r\n\r\nEmpresa Contratante...,6048,AGB: Contratacao de pessoa juridica para plano...,agb contrataca pesso jurid plan municip saneam...
7901,distribuidor de material medico hospitalar. Ca...,7901,Vaga para Assistente Administrativo,vag assistent administr
1082,"Revender cosmeticos o boticario, indicar novos...",1082,Vaga para Revendedor,vag revendedor
8685,Efetuar a inspecao diaria de itens basicos de ...,8685,Motorista Categoria D,motor categor d
5541,Assistente fiscal ensino superior cursando ou ...,5541,Vaga para Assistente Fiscal,vag assistent fiscal
1912,Auxiliar de servicos gerais rh 10 Contrata par...,1912,Vaga para Auxiliar de Servicos Gerais,vag auxili servic ger
1157,Empresa em indaiatubaEsta com 1Vaga(s) em aber...,1157,Vaga para Mecanico de Manutencao,vag mecan manutenca


## A.5 Tokenize the description

In [5]:
df['token_description'] = df['description'].transform(tokenfy)
# df.sample(10).head(10)
df.head(10)

,description,id,title,token_title,token_description
0,Auxilia nas rotinas do departamento administra...,0,Aprendiz Administrativo,aprendiz administr,auxil rotin depart administr acompanh relatori...
1,\r\n Sera responsavel pelo dese...,1,Programador(a),program,ser responsavel desenvolv manutenca sit loj v...
2,Sera responsavel pelo desenvolvimento e manute...,2,Programador(a),program,ser responsavel desenvolv manutenca sit loj vi...
3,Realizar conciliacao de contas contabeis. Anal...,3,Estagiario(a) Contabilidade,estagiario contabil,realiz conciliaca cont contab analis demonstra...
4,Realizar conciliacao de contas contabeis. Anal...,4,Estagiario(a) Contabilidade,estagiario contabil,realiz conciliaca cont contab analis demonstra...
5,Sera responsavel pelo desenvolvimento e manute...,5,Programador(a),program,ser responsavel desenvolv manutenca sit loj vi...
6,Realizar conciliacao de contas contabeis. Anal...,6,Estagiario(a) Contabilidade,estagiario contabil,realiz conciliaca cont contab analis demonstra...
7,Realizar conciliacao de contas contabeis. Anal...,7,Estagiario(a) Contabilidade,estagiario contabil,realiz conciliaca cont contab analis demonstra...
8,Sera responsavel pelo desenvolvimento e manute...,8,Programador(a),program,ser responsavel desenvolv manutenca sit loj vi...
9,A Empresa Tok&Stok conta com mais de 4.000 fun...,9,Vendedor Cadista - Copacabana,vendedor cadist copacaban,empres tokstok cont 4000 funcionari distribu 5...


## A.6 Token 2 idx

In [6]:
word2idx={}
df = data2idx(df, word2idx)
# df.sample(10).head(10)
df.head(10)

document:9997 of 9998	VOCAB:25671	WORD COUNT:574479	


,description,id,title,token_title,idx_description
0,Auxilia nas rotinas do departamento administra...,0,Aprendiz Administrativo,aprendiz administr,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17
1,\r\n Sera responsavel pelo dese...,1,Programador(a),program,18 19 20 21 22 23 24 25 26 27 28 29 21 30 22 3...
2,Sera responsavel pelo desenvolvimento e manute...,2,Programador(a),program,19 20 21 22 23 24 25 26 27 28 29 21 30 22 31 3...
3,Realizar conciliacao de contas contabeis. Anal...,3,Estagiario(a) Contabilidade,estagiario contabil,40 41 42 43 44 45 46 47 7 48 49 50 46
4,Realizar conciliacao de contas contabeis. Anal...,4,Estagiario(a) Contabilidade,estagiario contabil,40 41 42 43 44 45 46 47 7 48 49 50 46
5,Sera responsavel pelo desenvolvimento e manute...,5,Programador(a),program,19 20 21 22 23 24 25 26 27 28 29 21 30 22 31 3...
6,Realizar conciliacao de contas contabeis. Anal...,6,Estagiario(a) Contabilidade,estagiario contabil,40 41 42 43 44 45 46 47 7 48 49 50 46
7,Realizar conciliacao de contas contabeis. Anal...,7,Estagiario(a) Contabilidade,estagiario contabil,40 41 42 43 44 45 46 47 7 48 49 50 46
8,Sera responsavel pelo desenvolvimento e manute...,8,Programador(a),program,19 20 21 22 23 24 25 26 27 28 29 21 30 22 31 3...
9,A Empresa Tok&Stok conta com mais de 4.000 fun...,9,Vendedor Cadista - Copacabana,vendedor cadist copacaban,51 52 42 53 54 55 56 24 57 58 59 60 61 62 63 6...


# Titles

## 1. Distinct

In [5]:
distinct_titles= set(df['title'])
distinct_token_titles= set(df['token_title'])
print('distinct titles %d' % (len(distinct_titles)))
print('distinct token titles %d' % (len(distinct_token_titles)))

distinct titles 3804
distinct token titles 3276


## 2. Frequency

In [6]:
dfpivot= df.pivot_table(index='token_title',values='title', aggfunc=len)
print('top 20 job postings')
dfpivot.sort_values('title',ascending=False).head(20)

top 20 job postings


,title
token_title,
vag vendedor,171
vag estagiari,137
vendedor,96
vag consultor vend,76
vag auxili administr,66
vendedor extern,65
vag atendent,64
vag vendedor extern,63
vag auxili servic ger,57
